In [1]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
tqdm.pandas()

In [2]:
hosts = pd.read_csv("data/host.csv", header=None)
hosts.columns = ["url"]

In [3]:
hosts.head()

,url
0,api.youla.io
1,favicon.yandex.net
2,w-74721.fp.kaspersky-labs.com
3,questtime.net
4,passport-authproxy.taxi.yandex.net


In [4]:
start_with_api = hosts["url"].str.contains("^api", regex=True).astype(int)
sum(start_with_api)

25037

In [5]:
hosts["start_with_api"] = start_with_api

In [6]:
hosts.head()

,url,start_with_api
0,api.youla.io,1
1,favicon.yandex.net,0
2,w-74721.fp.kaspersky-labs.com,0
3,questtime.net,0
4,passport-authproxy.taxi.yandex.net,0


In [53]:
status_codes = []
content_types = []
is_json = []
is_redirect = []
size_of_cookies = []
encodings = []

In [62]:
def process_url(url):
    try:
        response = requests.get("http://"+url, timeout=0.5)
        
        try: 
            status_codes.append(response.status_code)
        except:
            status_codes.append(-1)
        
        try:
            content_types.append(response.headers["Content-Type"].split(";")[0])
        except:
            content_types.append("none")
            
        try: 
            response.json()
            is_json.append(1)
        except:
            is_json.append(0)
            
        try:
            is_redirect.append(int(response.is_redirect))
        except:
            is_redirect.append(-1)
            
        try:
            size_of_cookies.append(len(response.cookies))
        except:
            size_of_cookies.append(-1)
            
        try:
            encodings.append(response.encoding if len(response.encoding) != 0 else "none")
        except: 
            encodings.append("none")
        
    except:
        status_codes.append(-1)
        content_types.append("none")
        is_json.append(-1)
        is_redirect.append(-1)
        size_of_cookies.append(-1)
        encodings.append("none")

In [63]:
hosts["url"][:50].progress_apply(process_url) # example
print("Done")

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:11<00:00,  2.61it/s]

Done


In [64]:
content_types[:10]

['text/html',
 'none',
 'none',
 'text/html',
 'application/json',
 'text/html',
 'none',
 'text/html',
 'text/plain',
 'application/json']

In [65]:
status_codes[:10]

[200, 404, -1, 200, 404, 200, -1, 404, 204, 403]

In [66]:
is_json[:10]

[0, 0, -1, 0, 1, 0, -1, 0, 0, 1]

In [67]:
is_redirect[:10]

[0, 0, -1, 0, 0, 0, -1, 0, 0, 0]

In [68]:
encodings[:10]

['UTF-8',
 'none',
 'none',
 'UTF-8',
 'utf-8',
 'utf-8',
 'none',
 'UTF-8',
 'ISO-8859-1',
 'utf-8']

In [69]:
size_of_cookies[:10]

[0, 0, -1, 1, 0, 5, -1, 0, 0, 0]